# Problemas de Satisfacción de Restricciones
- Ricardo Méndez 21289
- Sara Echeverría 21371
- Francisco Castillo 21562
- Repositorio de Github: https://github.com/bl33h/constraintSatisfactionProblems

# Task 1 - Teoría

## Algoritmo AC-3 y su relación con el algoritmo de backtracking search
El algoritmo AC-3 optimiza la resolución de problemas de satisfacción de restricciones (CSP) eliminando previamente valores que no pueden satisfacer las restricciones entre variables, reduciendo así el espacio de búsqueda. Por su parte, el algoritmo de backtracking explora este espacio simplificado para encontrar soluciones, retrocediendo cuando una ruta no conduce a una solución válida. Su relación radica en la combinación de estos, pues dan como resultado una estrategia eficaz en la cual AC-3 reduce el espacio de posibilidades y el algoritmo de backtracking navega por este espacio optimizado en busca de la solución (Gao, 2024).


## Arc Consistency
El término de ‘Arc Consistency’ describe un método eficaz para abordar problemas de satisfacción de restricciones, mediante el cual se eliminan las opciones inviables que no se ajustan a las restricciones establecidas (Brown, 2010).

## Referencias
- Brown. (2010). CSPs: Arc Consistency. The University of British Columbia. https://www.cs.ubc.ca/~kevinlb/teaching/cs322%20-%202009-10/Lectures/CSP3.pdf
- Gao. (2024). Constraint Satisfaction Problems: Backtracking Search and Arc Consistency. University of Waterloo. https://cs.uwaterloo.ca/~a23gao/cs486686_s19/slides/lec05_csp_arc_consistency_backtracking_search_nosol.pdf

# Task 2

## Restricciones
- 4 estudiantes y 7 exámenes diferentes
- Todos los exámenes se realizan en tres días
- Ningún estudiante puede tener más de un exámen por día.
- Los estudiantes que toman el mismo curso no pueden tener exámenes programados para el mismo día

In [1]:
import time
import random

random.seed(123)

In [2]:
class MeasureTime:
    def __init__(self, ):
        self.start = 0
        self.end = 0

    def __enter__(self):
        self.start = time.time()
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = time.time()
        print(f'Time elapsed: {self.end - self.start} seconds')
        
    def get_time(self):
        return self.end - self.start

In [3]:
measure_time = MeasureTime()

In [4]:
results = {
    'backtracking': {
        'time': None,
        'solution': None
    },
    'beam_search': {
        'time': None,
        'solution': None
    },
    'local_search': {
        'time': None,
        'solution': None
    }
}

## Definición de Valores

In [5]:
exams = ['Math', 'Physics', 'Chemistry', 'History', 'Biology', 'Geography', 'Literature']
days = ['Monday', 'Tuesday', 'Wednesday']
students_names = ['Francisco', 'Sara', 'Ricardo', 'Fulanito']

In [6]:
class Student:
    def __init__(self, name):
        self.name = name
        self.exams = {'None'}

    def add_exam(self, exam):
        self.exams.add(exam)

    def __str__(self):
        return self.name

    def __repr__(self):
        return self.name

    def __eq__(self, other):
        return self.name == other.name

    def __hash__(self):
        return hash(self.name)

### Creación de estudiantes y asignación de exámenes (para cumplir con las restricciones)

In [7]:
# Create Students
students = []
for name in students_names:
    students.append(Student(name))

# Assign Exams to Students
for i in range(len(exams)):
    students[i % len(students)].add_exam(exams[i])

# Assign Exams until each student has 3 different exams
for student in students:
    while len(student.exams) < 4:
        additional_exam = random.choice(exams)
        student.add_exam(additional_exam)

# Print students and their exams
print('Students and their exams:')
for student in students:
    print(f'{student}: {[exam for exam in student.exams]}')

Students and their exams:
Francisco: ['Biology', 'Math', 'None', 'Chemistry']
Sara: ['Geography', 'Physics', 'None', 'Math']
Ricardo: ['History', 'Literature', 'None', 'Chemistry']
Fulanito: ['History', 'Math', 'None', 'Chemistry']


### Definición de Variables, Dominios y Restricciones

In [8]:
variables = []
for student in students:
    for day in days:
        variables.append((student, day))

domains = {}
for var in variables:
    domains[var] = [exam for exam in var[0].exams]
    
print('Domains:')
domains

Domains:


{(Francisco, 'Monday'): ['Biology', 'Math', 'None', 'Chemistry'],
 (Francisco, 'Tuesday'): ['Biology', 'Math', 'None', 'Chemistry'],
 (Francisco, 'Wednesday'): ['Biology', 'Math', 'None', 'Chemistry'],
 (Sara, 'Monday'): ['Geography', 'Physics', 'None', 'Math'],
 (Sara, 'Tuesday'): ['Geography', 'Physics', 'None', 'Math'],
 (Sara, 'Wednesday'): ['Geography', 'Physics', 'None', 'Math'],
 (Ricardo, 'Monday'): ['History', 'Literature', 'None', 'Chemistry'],
 (Ricardo, 'Tuesday'): ['History', 'Literature', 'None', 'Chemistry'],
 (Ricardo, 'Wednesday'): ['History', 'Literature', 'None', 'Chemistry'],
 (Fulanito, 'Monday'): ['History', 'Math', 'None', 'Chemistry'],
 (Fulanito, 'Tuesday'): ['History', 'Math', 'None', 'Chemistry'],
 (Fulanito, 'Wednesday'): ['History', 'Math', 'None', 'Chemistry']}

In [9]:
# Constraints
# NOTE THAT, BY THE NATURE OF THE ASSIGNMENT, THE STUDENT CAN'T TAKE MORE THAN ONE EXAM PER DAY
def constraint_all_exams_taken(assignment: dict) -> bool:
    # All values of the exams list must be in the values of the dict
    exams_taken = []
    for var in assignment:
        if assignment[var] != 'None':
            exams_taken.append(assignment[var])

    return set(exams) == set(exams_taken)  # If all the exams in the list are part of the assignment, return True

In [10]:
def constraint_one_exam_type_per_day(assignment: dict) -> bool:
    # At max one exam of each type per day
    exams_taken = {}  # day: list[exams]
    for day in days:
        exams_taken[day] = []
    for var in assignment:
        if assignment[var] != 'None':
            if assignment[var] in exams_taken[var[1]]:
                return False
            exams_taken[var[1]].append(assignment[var])
    return True

In [11]:
def print_result(result: dict):
    # Sort by day
    sorted_result = sorted(result.items(), key=lambda x: x[0][1])
    for student_day, exam in sorted_result:
        if exam == 'None':
            continue
        student, day = student_day
        print(f'{day}:\t\t{student} - {exam}')

## Backtracking Search 

In [12]:
def backtracking_search(domains):
    def recursive_backtracking(assignment):
        if len(assignment) == len(domains):
            return assignment

        unassigned = [v for v in domains.keys() if v not in assignment]
        first = unassigned[0]
        for value in domains[first]:
            local_assignment = assignment.copy()
            local_assignment[first] = value
            if constraint_one_exam_type_per_day(local_assignment):
                result = recursive_backtracking(local_assignment)
                if result is not None:
                    # If the result contains all the exams
                    if constraint_all_exams_taken(result):
                        return result
        return None

    return recursive_backtracking({})

In [13]:
with measure_time:
    results['backtracking']['solution'] = backtracking_search(domains)
    
results['backtracking']['time'] = measure_time.get_time()

Time elapsed: 0.06699252128601074 seconds


In [14]:
print_result(results['backtracking']['solution'])
print(f"Time elapsed: {results['backtracking']['time']} seconds")

Monday:		Francisco - Biology
Monday:		Sara - Geography
Monday:		Ricardo - History
Monday:		Fulanito - Math
Tuesday:		Francisco - Biology
Tuesday:		Sara - Geography
Tuesday:		Ricardo - History
Tuesday:		Fulanito - Math
Wednesday:		Francisco - Biology
Wednesday:		Sara - Physics
Wednesday:		Ricardo - Literature
Wednesday:		Fulanito - Chemistry
Time elapsed: 0.06699252128601074 seconds


## Beam Search

In [15]:
import random

def beam_search(domains, beam_width):
    def evaluate_assignment(assignment):
        if constraint_all_exams_taken(assignment):
            return len(set(assignment.values()))
        return 0

    def generate_neighbors(assignment):
        neighbors = []
        for var in domains.keys():
            for value in domains[var]:
                neighbor = assignment.copy()
                neighbor[var] = value
                if constraint_one_exam_type_per_day(neighbor):
                    neighbors.append(neighbor)
        return neighbors

    def select_best_neighbors(neighbors, beam_width):
        neighbors.sort(key=lambda x: evaluate_assignment(x), reverse=True)
        return neighbors[:beam_width]

    current_assignment = {}
    for var in domains.keys():
        choice = random.choice(domains[var])
        student_assigned = False
        for assignment in current_assignment:
            if current_assignment[assignment] == choice:
                student_assigned = True
                break
        if not student_assigned:
            current_assignment[var] = random.choice(domains[var])
        else:
            current_assignment[var] = 'None'

    while not constraint_all_exams_taken(current_assignment):
        neighbors = generate_neighbors(current_assignment)
        current_assignment = random.choice(select_best_neighbors(neighbors, beam_width))

    return current_assignment

In [44]:
with measure_time:
    beam_width = 12
    result = beam_search(domains, beam_width)
    results['beam_search']['solution'] = result
    print_result(result)
results['beam_search']['time'] = measure_time.get_time()


Monday:		Francisco - Chemistry
Monday:		Sara - Geography
Monday:		Fulanito - History
Tuesday:		Francisco - Biology
Tuesday:		Fulanito - Math
Wednesday:		Sara - Physics
Wednesday:		Ricardo - Literature
Time elapsed: 0.022001028060913086 seconds


## Local Search

In [45]:
def local_search(domains, max_iterations=1000):
    # assign a random exam from their domain to each student for each day
    assignment = {var: random.choice(domains[var]) for var in domains}
    
    for _ in range(max_iterations):
        # check if current assignment satisfies all constraints
        if constraint_all_exams_taken(assignment) and constraint_one_exam_type_per_day(assignment):
            return assignment
        
        # select a random variable to try and improve
        var_to_improve = random.choice(list(assignment.keys()))
        current_exam = assignment[var_to_improve]
        
        # try to find a better value for the selected variable
        for potential_exam in domains[var_to_improve]:
            if potential_exam == current_exam:
                continue
            
            # create a new assignment with the potential change
            new_assignment = assignment.copy()
            new_assignment[var_to_improve] = potential_exam
            
            # accept the new assignment if it improves or maintains the constraint satisfaction
            if constraint_one_exam_type_per_day(new_assignment):
                assignment = new_assignment
                break  # stop looking for improvements for this iteration
        
    return None

with measure_time:
    solution = local_search(domains)

if solution:
    print_result(solution)
    results['local_search']['solution'] = solution
    results['local_search']['time'] = measure_time.get_time()
else:
    print("No valid scheduling found.")

Time elapsed: 0.0 seconds
Monday:		Francisco - Chemistry
Monday:		Sara - Geography
Monday:		Ricardo - Literature
Tuesday:		Francisco - Biology
Tuesday:		Sara - Physics
Tuesday:		Ricardo - History
Tuesday:		Fulanito - Math
Wednesday:		Sara - Physics
Wednesday:		Fulanito - Math


In [21]:
with measure_time:
    results['local_search']['solution'] = local_search(domains)
    
results['local_search']['time'] = measure_time.get_time()

Time elapsed: 0.0 seconds


# Conclusiones
De cada uno de los algoritmos implementados, tome el tiempo que le toma encontrar una solución, y compare no
solo el tiempo, sino también la solución encontrada de cada uno. 

## Tiempos

In [46]:
results

{'backtracking': {'time': 0.06699252128601074,
  'solution': {(Francisco, 'Monday'): 'Biology',
   (Francisco, 'Tuesday'): 'Biology',
   (Francisco, 'Wednesday'): 'Biology',
   (Sara, 'Monday'): 'Geography',
   (Sara, 'Tuesday'): 'Geography',
   (Sara, 'Wednesday'): 'Physics',
   (Ricardo, 'Monday'): 'History',
   (Ricardo, 'Tuesday'): 'History',
   (Ricardo, 'Wednesday'): 'Literature',
   (Fulanito, 'Monday'): 'Math',
   (Fulanito, 'Tuesday'): 'Math',
   (Fulanito, 'Wednesday'): 'Chemistry'}},
 'beam_search': {'time': 0.022001028060913086,
  'solution': {(Francisco, 'Monday'): 'Chemistry',
   (Francisco, 'Tuesday'): 'Biology',
   (Francisco, 'Wednesday'): 'None',
   (Sara, 'Monday'): 'Geography',
   (Sara, 'Tuesday'): 'None',
   (Sara, 'Wednesday'): 'Physics',
   (Ricardo, 'Monday'): 'None',
   (Ricardo, 'Tuesday'): 'None',
   (Ricardo, 'Wednesday'): 'Literature',
   (Fulanito, 'Monday'): 'History',
   (Fulanito, 'Tuesday'): 'Math',
   (Fulanito, 'Wednesday'): 'None'}},
 'local_search

In [47]:
for key in results:
    if results[key]['solution'] is None:
        print(f'{key}: No solution found')
        continue
    print(f'{key}: {results[key]["time"]} seconds', end='\n\n')
    print_result(results[key]['solution'])
    print("=====================================")

backtracking: 0.06699252128601074 seconds

Monday:		Francisco - Biology
Monday:		Sara - Geography
Monday:		Ricardo - History
Monday:		Fulanito - Math
Tuesday:		Francisco - Biology
Tuesday:		Sara - Geography
Tuesday:		Ricardo - History
Tuesday:		Fulanito - Math
Wednesday:		Francisco - Biology
Wednesday:		Sara - Physics
Wednesday:		Ricardo - Literature
Wednesday:		Fulanito - Chemistry
beam_search: 0.022001028060913086 seconds

Monday:		Francisco - Chemistry
Monday:		Sara - Geography
Monday:		Fulanito - History
Tuesday:		Francisco - Biology
Tuesday:		Fulanito - Math
Wednesday:		Sara - Physics
Wednesday:		Ricardo - Literature
local_search: 0.0 seconds

Monday:		Francisco - Chemistry
Monday:		Sara - Geography
Monday:		Ricardo - Literature
Tuesday:		Francisco - Biology
Tuesday:		Sara - Physics
Tuesday:		Ricardo - History
Tuesday:		Fulanito - Math
Wednesday:		Sara - Physics
Wednesday:		Fulanito - Math


#### Como se puede ver en los resultados de las dos celdas anteriores, backtracking es el algoritmo al que más tiempo le tomó encontrar una asignación, mientras que local search fue el más rápido. Cabe recalcar que tanto local search como beam search son algoritmos que se pueden considerar high risk - high reward, pues dependen mucho del estado incial que se plantee. Un estado inicial ineficiente puede llevar a tiempos de espera muy altos o directamente a una respuesta inválida.